# Tutorial notebook for working Planetary Computer

In [1]:
import sys

# add the src directory to the path so that we can import generic functions
sys.path.insert(0, "src")

import logging
import os
import pathlib

# Sometimes you need libraries which are not included in the planetary computer image. That's
# generally not a problem because you can install them with pip. 
os.system("pip install python-dotenv -q")

import dask_geopandas
import geopandas as gpd
import hvplot.pandas
import pandas as pd
import pystac
import shapely
from dotenv import load_dotenv
from ipyleaflet import Map, basemaps

# load environment variables
load_dotenv(override=True)

# tokens to access data in private containers 
sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
coclico_storage_options = {"account_name": "coclico", "credential": sas_token}

# disable logging messages from azure 
logging.getLogger("azure").setLevel(logging.WARNING)

## Load from STAC catalog

Load the transects from our CoCliCo STAC catalog. 

In [2]:
coclico_catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)

In [3]:
coclico_catalog

id: coclico-catalog
title: CoCliCo STAC Catalog
description: CoCliCo Catalog
type: Catalog
"assets: {'metadata': {'href': 'https://www.deltares.nl/en/news/development-of-a-new-european-platform-on-climate-services-for-coastal-risks-and-adaptation/', 'type': 'application/html', 'title': 'Deltares Website', 'description': 'Deltares Website about CoCliCo', 'roles': ['metadata']}}"
"summaries: {'keywords': ['Sea Levels', 'Natural Hazards', 'Exposure & Vulnerability', 'Risk & Adaptation']}"
"extent: {'spatial': {'bbox': [[-180, -90, 180, 90]]}, 'temporal': {'interval': [['2015-10-22T00:00:00Z', None]]}}"
license: various
id: ssl
title: Extreme surge level
"description: Dataset with extreme Storm Surge Levels (SSL) at the European scale. SSL are estimated for three climate scenarios (Historical, RCP4.5 and RCP8.5) for eight return periods (5, 10, 20, 50, 100, 200, 500 and 1000) according to the Peak Over Threshold method. This dataset is part of the [LISCOAST](https://data.jrc.ec.europa.eu/collection/LISCOAST) project. See this [article](https://doi.org/10.1007/s00382-016-3019-5) for more dataset-specific information."


In [4]:
list(coclico_catalog.get_all_collections())

[<Collection id=ssl>,
 <Collection id=wef>,
 <Collection id=eesl>,
 <Collection id=floodmaps>,
 <Collection id=sc>,
 <Collection id=cbca>,
 <Collection id=cfr>,
 <Collection id=smd>,
 <Collection id=cisi>,
 <Collection id=slp5>,
 <Collection id=slp6>,
 <Collection id=slp6_pilot>,
 <Collection id=coastal-mask>,
 <Collection id=shorelinemonitor-shorelines>,
 <Collection id=gcts-2000m>]

In [5]:
gcts = coclico_catalog.get_child("gcts-2000m")
gcts

id: gcts-2000m
title: Global Coastal Transect System 2000 m
"description: This Global Coastal Transect System (GCTS-2000m) consists of transects that are 2000 m long, derived from a generalized (zoom level 9) OpenStreetMap coastline (2022-02-22). The transects are placed at equally-spaced 100 m alongshore resolution."
"providers: Deltares (producer, processor, host, licensor)"
type: Collection
"item_assets: {'data': {'title': 'Coastal Transects', 'description': 'Coastal transects (2000 m) as LineString geometries in Parquet format.', 'roles': ['data'], 'type': 'application/vnd.apache.parquet', 'table:storage_options': {'account_name': 'coclico'}, 'table:columns': [{'name': 'tr_name', 'type': 'string', 'description': 'Name of the transect, unique identifier that is composed by the source coastline and the interpolated distance to the transect origin.'}, {'name': 'lon', 'type': 'float', 'description': 'Longitude of the transect starting point.'}, {'name': 'lat', 'type': 'float', 'description': 'Latitude of the transect starting point.'}, {'name': 'bearing', 'type': 'float', 'description': 'North bearing of the transect from the landward side in degrees, with north as the reference.'}, {'name': 'utm_crs', 'type': 'int32', 'description': 'EPSG code representing the UTM Coordinate Reference System for the transect.'}, {'name': 'coastline_name', 'type': 'int32', 'description': 'Identifier of the coastline from which the transect was derived.'}, {'name': 'geometry', 'type': 'byte_array', 'description': 'Well-Known Binary (WKB) representation of the transect as a linestring geometry.'}, {'name': 'bbox', 'type': 'struct', 'description': 'Bounding box of the transect geometry, given by minimum and maximum coordinates in x (longitude) and y (latitude).'}, {'name': 'quadkey', 'type': 'string', 'description': 'QuadKey corresponding to the transect origin location at zoom 12, following the Bing Maps Tile System for spatial indexing.'}, {'name': 'isoCountryCodeAlpha2', 'type': 'string', 'description': 'ISO 3166-1 alpha-2 country code for the country in which the transect is located.'}, {'name': 'admin_level_1_name', 'type': 'string', 'description': 'Name of the first-level administrative division (e.g., country) in which the transect is located.'}, {'name': 'isoSubCountryCode', 'type': 'string', 'description': 'ISO code for the sub-country or second-level administrative division in which the transect is located.'}, {'name': 'admin_level_2_name', 'type': 'string', 'description': 'Name of the second-level administrative division (e.g., state or province) in which the transect is located.'}, {'name': 'bounding_quadkey', 'type': 'string', 'description': 'QuadKey of the bounding quadtile that encapsulates the entire transect geometry, used for spatial indexing at a broader scale.'}]}}"
base_url: az://transects/gcts-2000m.parquet
"sci:citation: Calkoen et al., 2023, in progress."
version: 1
https://stac-extensions.github.io/item-assets/v1.0.0/schema.json
https://stac-extensions.github.io/scientific/v1.0.0/schema.json


### Use a dynamic map to extract data by region of interest

The IPyleaflet map below can be used to find the bbox coordinates of a certain region.
Zoom to the area where you want to extract data and run the next cell. Please keep in
mind to wait 1 second because the map has to be rendered before the coordinates can be
extracted. 

In [6]:
m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
m.center = 43.406241, -2.976665
m.zoom = 9
m.layout.height = "800px"
m

Map(center=[43.406241, -2.976665], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

## IMPORTANT NOTE: Wait for the map to render before you run the next cell

rendering the map takes a second, so you need to pause 1 second before running the next cell otherwise you cannot parse the north/west/east/south bounds

## Import functions from project directory 

In [13]:
from bilbao.utils import geo_bbox
# this makes a GeoPandas dataframe from the DynamicMap that is rendered above
roi = geo_bbox(m.west, m.south, m.east, m.north)
roi.explore()

In [14]:
# makes a list of all items (data partitions) in the GCTS STAC catalog
items = list(gcts.get_all_items())

## The dataset is partitioned into geospatial chunks

The dataset is divided into different chunks, that each span a different region of the world. In the next cell
we read the spatial extends of each chunk and compose that into a GeoDataFrame

In [15]:
bboxes = pd.concat([geo_bbox(*i.to_dict()["bbox"]) for i in items])
bboxes = bboxes.reset_index(drop=True)
bboxes.explore()

## Now we can find the bboxes that cover our region of interest

In [16]:
bboxes_roi = gpd.sjoin(bboxes, roi)[bboxes.columns]
items_roi = [items[i] for i in bboxes_roi.index]

In [17]:
items_roi

[<Item id=minx_-90.02_miny_-0.01_part_0>]

In [18]:
items_roi[0]

id: minx_-90.02_miny_-0.01_part_0
"bbox: [-90.01697414042903, -0.00870772774355085, -1.175311021069507e-05, 66.52205836170414]"
title: Coastal Transects
description: Parquet dataset with the coastal transects
"table:columns: [{'name': 'tr_name', 'type': 'string', 'description': 'Name of the transect, unique identifier that is composed by the source coastline and the interpolated distance to the transect origin.'}, {'name': 'lon', 'type': 'float', 'description': 'Longitude of the transect starting point.'}, {'name': 'lat', 'type': 'float', 'description': 'Latitude of the transect starting point.'}, {'name': 'bearing', 'type': 'float', 'description': 'North bearing of the transect from the landward side in degrees, with north as the reference.'}, {'name': 'utm_crs', 'type': 'int32', 'description': 'EPSG code representing the UTM Coordinate Reference System for the transect.'}, {'name': 'coastline_name', 'type': 'int32', 'description': 'Identifier of the coastline from which the transect was derived.'}, {'name': 'geometry', 'type': 'binary', 'description': 'Well-Known Binary (WKB) representation of the transect as a linestring geometry.'}, {'name': 'bbox.maxx', 'type': 'double'}, {'name': 'bbox.maxy', 'type': 'double'}, {'name': 'bbox.minx', 'type': 'double'}, {'name': 'bbox.miny', 'type': 'double'}, {'name': 'quadkey', 'type': 'string', 'description': 'QuadKey corresponding to the transect origin location at zoom 12, following the Bing Maps Tile System for spatial indexing.'}, {'name': 'isoCountryCodeAlpha2', 'type': 'string', 'description': 'ISO 3166-1 alpha-2 country code for the country in which the transect is located.'}, {'name': 'admin_level_1_name', 'type': 'string', 'description': 'Name of the first-level administrative division (e.g., country) in which the transect is located.'}, {'name': 'isoSubCountryCode', 'type': 'string', 'description': 'ISO code for the sub-country or second-level administrative division in which the transect is located.'}, {'name': 'admin_level_2_name', 'type': 'string', 'description': 'Name of the second-level administrative division (e.g., state or province) in which the transect is located.'}, {'name': 'bounding_quadkey', 'type': 'string', 'description': 'QuadKey of the bounding quadtile that encapsulates the entire transect geometry, used for spatial indexing at a broader scale.'}]"
"proj:bbox: [-90.01697414042903, -0.00870772774355085, -1.175311021069507e-05, 66.52205836170414]"
proj:epsg: 4326
table:row_count: 2227837
datetime: 2024-02-28T18:36:40.573348Z
https://stac-extensions.github.io/table/v1.2.0/schema.json
https://stac-extensions.github.io/projection/v1.1.0/schema.json


## The STAC items contain references to where the data is stored

In [19]:
hrefs = [i.assets["data"].href for i in items_roi]

## Cloud based data

The href that you see below is a url to a cloud bucket with the transects for the area of interest. The prefix "az://" is the protocol for Azure cloud storage. So if you don't have a STAC catalog write out the href's yourself. 

In [20]:
hrefs

['az://transects/gcts-2000m.parquet/minx_-90.02_miny_-0.01_part_0.parquet']

## Reading the transect partitions that span our region of interest 

We will read the data from cloud storage - but only the data that spans our region of interest (the DynamicMap above). 

## Dask dataframes are lazy

These dataframes are not in memory yet. We still have to trigger the compute (see cell below)

In [21]:
dask_geopandas.read_parquet(hrefs, storage_options=coclico_storage_options)

,tr_name,lon,lat,bearing,utm_crs,coastline_name,geometry,bbox,quadkey,isoCountryCodeAlpha2,admin_level_1_name,isoSubCountryCode,admin_level_2_name,bounding_quadkey
npartitions=1,,,,,,,,,,,,,,
,object,float32,float32,float32,int32,int32,geometry,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


## Compute the transects that span our region of interest

The transects are not in memory yet. In the next cell we will trigger the retrieval from cloud storage to local client by doing a `ddf.compute()` call. Note that we can also mix in regular Pandas operations, like sorting. Currently the transects are sorted by QuadKey to optimize fast read access by filter pushdown. If we want them sorted along the coastline we can do that by sorting the tr_name. 

In [22]:
%%time
transects = dask_geopandas.read_parquet(hrefs, storage_options=coclico_storage_options)
transects = (
    transects.sjoin(
        dask_geopandas.from_geopandas(roi.to_crs(transects.crs), npartitions=1)
    )
    .drop(columns=["index_right"])
    .sort_values("tr_name")
    .compute()
)

CPU times: user 9.52 s, sys: 2.44 s, total: 12 s
Wall time: 13.4 s


## Show the first lines of the table

In [25]:
transects.head()

,tr_name,lon,lat,bearing,utm_crs,coastline_name,geometry,bbox,quadkey,isoCountryCodeAlpha2,admin_level_1_name,isoSubCountryCode,admin_level_2_name,bounding_quadkey
1528900,cl33090tr02875265,-1.526899,43.522533,293.163147,32630,33090,"LINESTRING (-1.51561 43.51885, -1.53819 43.52622)","{'maxx': -1.5156133084939416, 'maxy': 43.52622...",031132311113,FR,France,FR-NAQ,Nouvelle-Aquitaine,031333123110
1527268,cl33090tr02875365,-1.527412,43.521713,294.044983,32630,33090,"LINESTRING (-1.51621 43.51790, -1.53862 43.52553)","{'maxx': -1.516205931128326, 'maxy': 43.525527...",031132311112,FR,France,FR-NAQ,Nouvelle-Aquitaine,031333123110
1527266,cl33090tr02875465,-1.527936,43.520901,294.044983,32630,33090,"LINESTRING (-1.51673 43.51709, -1.53914 43.52471)","{'maxx': -1.5167300487367181, 'maxy': 43.52471...",031132311112,FR,France,FR-NAQ,Nouvelle-Aquitaine,031333123110
1523374,cl33090tr02875565,-1.528464,43.520084,294.721710,32630,33090,"LINESTRING (-1.51732 43.51617, -1.53961 43.52399)","{'maxx': -1.5173203258313708, 'maxy': 43.52399...",031132311110,FR,France,FR-NAQ,Nouvelle-Aquitaine,03133312311
1523346,cl33090tr02875665,-1.529001,43.519272,294.721710,32630,33090,"LINESTRING (-1.51786 43.51536, -1.54015 43.52318)","{'maxx': -1.5178576114348519, 'maxy': 43.52318...",031132311110,FR,France,FR-NAQ,Nouvelle-Aquitaine,03133312311


## Holoviews interactive visualization

In [24]:
import colorcet as cc
import hvplot.pandas

transects_plot = transects[["geometry", "bearing"]].sample(500).hvplot(
    geo=True,
    tiles="ESRI",
    color="bearing",
    cmap=cc.CET_C10,
    width=700,
    height=500,
    clabel="North bearing [deg]",    
    xlabel="Longitude [deg]",
    ylabel="Latitude [deg]",
    title="Cross-shore transects (100m alongshore), Euskadi.",
    colorbar=True,
    tools=["wheel_zoom"]
)
transects_plot

:Overlay
   .Tiles.I :Tiles   [x,y]
   .Path.I  :Path   [Longitude,Latitude]   (bearing)

## Dask Cluster

In [40]:
import dask_gateway

cluster = dask_gateway.GatewayCluster()
client = cluster.get_client()
cluster.adapt(minimum=2, maximum=50)
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: https://pccompute.westeurope.cloudapp.azure.com/compute/services/dask-gateway/clusters/prod.ec13e487b6904157a3f4a60706b32d37/status,


## Accessing ERA 5 for our region of interest

In [41]:
import pystac_client

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1/"
)

time_range = "2020-01-01/2023-12-31"
bbox = [m.west, m.south, m.east, m.north]

search = catalog.search(collections=["era5-pds"], bbox=bbox, datetime=time_range, query={"era5:kind": {"eq": "an"}})
items = search.get_all_items()
len(items)

/srv/conda/envs/notebook/lib/python3.11/site-packages/pystac_client/item_search.py:841: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(


12

In [42]:
item = items[0]
item

id: era5-pds-2020-12-an
"bbox: [-180, -90, 180, 90]"
datetime: None
era5:kind: an
end_datetime: 2020-12-31T23:00:00Z
"cube:variables: {'surface_air_pressure': {'type': 'data', 'unit': 'Pa', 'attrs': {'units': 'Pa', 'nameCDM': 'Surface_pressure_surface', 'long_name': 'Surface pressure', 'nameECMWF': 'Surface pressure', 'product_type': 'analysis', 'standard_name': 'surface_air_pressure', 'shortNameECMWF': 'sp'}, 'shape': [744, 721, 1440], 'dimensions': ['time', 'lat', 'lon'], 'description': 'Surface pressure'}, 'sea_surface_temperature': {'type': 'data', 'unit': 'K', 'attrs': {'units': 'K', 'nameCDM': 'Sea_surface_temperature_surface', 'long_name': 'Sea surface temperature', 'nameECMWF': 'Sea surface temperature', 'product_type': 'analysis', 'standard_name': 'sea_surface_temperature', 'shortNameECMWF': 'sst'}, 'shape': [744, 721, 1440], 'dimensions': ['time', 'lat', 'lon'], 'description': 'Sea surface temperature'}, 'eastward_wind_at_10_metres': {'type': 'data', 'unit': 'm s**-1', 'attrs': {'units': 'm s**-1', 'nameCDM': '10_metre_U_wind_component_surface', 'long_name': '10 metre U wind component', 'nameECMWF': '10 metre U wind component', 'product_type': 'analysis', 'standard_name': 'eastward_wind', 'shortNameECMWF': '10u'}, 'shape': [744, 721, 1440], 'dimensions': ['time', 'lat', 'lon'], 'description': '10 metre U wind component'}, 'air_temperature_at_2_metres': {'type': 'data', 'unit': 'K', 'attrs': {'units': 'K', 'nameCDM': '2_metre_temperature_surface', 'long_name': '2 metre temperature', 'nameECMWF': '2 metre temperature', 'product_type': 'analysis', 'standard_name': 'air_temperature', 'shortNameECMWF': '2t'}, 'shape': [744, 721, 1440], 'dimensions': ['time', 'lat', 'lon'], 'description': '2 metre temperature'}, 'eastward_wind_at_100_metres': {'type': 'data', 'unit': 'm s**-1', 'attrs': {'units': 'm s**-1', 'nameCDM': '100_metre_U_wind_component_surface', 'long_name': '100 metre U wind component', 'nameECMWF': '100 metre U wind component', 'product_type': 'analysis', 'standard_name': 'eastward_wind', 'shortNameECMWF': '100u'}, 'shape': [744, 721, 1440], 'dimensions': ['time', 'lat', 'lon'], 'description': '100 metre U wind component'}, 'northward_wind_at_10_metres': {'type': 'data', 'unit': 'm s**-1', 'attrs': {'units': 'm s**-1', 'nameCDM': '10_metre_V_wind_component_surface', 'long_name': '10 metre V wind component', 'nameECMWF': '10 metre V wind component', 'product_type': 'analysis', 'standard_name': 'northward_wind', 'shortNameECMWF': '10v'}, 'shape': [744, 721, 1440], 'dimensions': ['time', 'lat', 'lon'], 'description': '10 metre V wind component'}, 'northward_wind_at_100_metres': {'type': 'data', 'unit': 'm s**-1', 'attrs': {'units': 'm s**-1', 'nameCDM': '100_metre_V_wind_component_surface', 'long_name': '100 metre V wind component', 'nameECMWF': '100 metre V wind component', 'product_type': 'analysis', 'standard_name': 'northward_wind', 'shortNameECMWF': '100v'}, 'shape': [744, 721, 1440], 'dimensions': ['time', 'lat', 'lon'], 'description': '100 metre V wind component'}, 'air_pressure_at_mean_sea_level': {'type': 'data', 'unit': 'Pa', 'attrs': {'units': 'Pa', 'nameCDM': 'Mean_sea_level_pressure_surface', 'long_name': 'Mean sea level pressure', 'nameECMWF': 'Mean sea level pressure', 'product_type': 'analysis', 'standard_name': 'air_pressure_at_mean_sea_level', 'shortNameECMWF': 'msl'}, 'shape': [744, 721, 1440], 'dimensions': ['time', 'lat', 'lon'], 'description': 'Mean sea level pressure'}, 'dew_point_temperature_at_2_metres': {'type': 'data', 'unit': 'K', 'attrs': {'units': 'K', 'nameCDM': '2_metre_dewpoint_temperature_surface', 'long_name': '2 metre dewpoint temperature', 'nameECMWF': '2 metre dewpoint temperature', 'product_type': 'analysis', 'standard_name': 'dew_point_temperature', 'shortNameECMWF': '2d'}, 'shape': [744, 721, 1440], 'dimensions': ['time', 'lat', 'lon'], 'description': '2 metre dewpoint temperature'}}"
start_datetime: 2020-12-01T00:00:00Z
"cube:dimensions: {'lat': {'axis': 'y', 'step': -0.25

In [43]:
import planetary_computer as pc
import xarray as xr

item = items[0]
signed_item = pc.sign(item)
datasets = [
    xr.open_dataset(asset.href, **asset.extra_fields["xarray:open_kwargs"])
    for asset in signed_item.assets.values()
]

ds = xr.combine_by_coords(datasets, join="exact")
ds

<xarray.Dataset>
Dimensions:                            (time: 744, lat: 721, lon: 1440)
Coordinates:
  * lat                                (lat) float32 90.0 89.75 ... -89.75 -90.0
  * lon                                (lon) float32 0.0 0.25 ... 359.5 359.8
  * time                               (time) datetime64[ns] 2020-12-01 ... 2...
Data variables:
    air_pressure_at_mean_sea_level     (time, lat, lon) float32 dask.array<chunksize=(372, 150, 150), meta=np.ndarray>
    air_temperature_at_2_metres        (time, lat, lon) float32 dask.array<chunksize=(372, 150, 150), meta=np.ndarray>
    dew_point_temperature_at_2_metres  (time, lat, lon) float32 dask.array<chunksize=(372, 150, 150), meta=np.ndarray>
    eastward_wind_at_100_metres        (time, lat, lon) float32 dask.array<chunksize=(372, 150, 150), meta=np.ndarray>
    eastward_wind_at_10_metres         (time, lat, lon) float32 dask.array<chunksize=(372, 150, 150), meta=np.ndarray>
    northward_wind_at_100_metres       (time, lat, lon) float32 dask.array<chunksize=(372, 150, 150), meta=np.ndarray>
    northward_wind_at_10_metres        (time, lat, lon) float32 dask.array<chunksize=(372, 150, 150), meta=np.ndarray>
    sea_surface_temperature            (time, lat, lon) float32 dask.array<chunksize=(372, 150, 150), meta=np.ndarray>
    surface_air_pressure               (time, lat, lon) float32 dask.array<chunksize=(372, 150, 150), meta=np.ndarray>
Attributes:
    institution:  ECMWF
    source:       Reanalysis
    title:        ERA5 forecasts

In [44]:
ds.air_pressure_at_mean_sea_level

<xarray.DataArray 'air_pressure_at_mean_sea_level' (time: 744, lat: 721,
                                                    lon: 1440)>
dask.array<open_dataset-air_pressure_at_mean_sea_level, shape=(744, 721, 1440), dtype=float32, chunksize=(372, 150, 150), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 90.0 89.75 89.5 89.25 ... -89.25 -89.5 -89.75 -90.0
  * lon      (lon) float32 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
  * time     (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
Attributes:
    long_name:       Mean sea level pressure
    nameCDM:         Mean_sea_level_pressure_surface
    nameECMWF:       Mean sea level pressure
    product_type:    analysis
    shortNameECMWF:  msl
    standard_name:   air_pressure_at_mean_sea_level
    units:           Pa

In [45]:
mean_air_pressure = ds.air_pressure_at_mean_sea_level.mean("time").compute()

In [58]:
import hvplot.xarray
import rioxarray
import numpy as np
mean_air_pressure = mean_air_pressure.rio.write_crs(4326).rio.write_nodata(np.nan)
mean_air_pressure.hvplot(x="lon", y="lat", geo=True, tiles="EsriImagery", rasterize=True, width=700, height=500, title="Mean Air Pressure", colorbar=True, clabel="Air Pressure [pa]")

:DynamicMap   []
   :Overlay
      .Tiles.I :Tiles   [x,y]
      .Image.I :Image   [lon,lat]   (air_pressure_at_mean_sea_level)